In [60]:
#!pip install -q tweepy
%reload_ext autoreload
%autoreload 2
import logging
import pandas as pd
import datetime
import tweepy
import json
from utils.extract import db_connection, download_data
logging.getLogger().setLevel(logging.WARNING)

In [61]:
#credenciales de db, twitter y emisoras
with open('creds.txt', encoding='utf-8') as data_file:
        creds = json.loads(data_file.read())

with open('tw_creds.txt', encoding='utf-8') as data_file:
    tw_creds = json.loads(data_file.read())

lista_cuentas = pd.read_csv('Cuentas BMBV.csv')



In [62]:
def new_tw_api(key_num):
    """
    Usa nuevas claves de twitter para hacer una nueva conexión al API
    """
    logging.warning('Nueva cuenta API: {}'.format(key_num))
    consumer_key = tw_creds['consumer_key' + str(key_num)]
    consumer_secret= tw_creds['consumer_secret' + str(key_num)]
    access_token= tw_creds['access_token' + str(key_num)]
    access_token_secret= tw_creds['access_token_secret' + str(key_num)]

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth)
    
    return api

In [63]:
# In this example, the handler is time.sleep(15 * 60),
# but you can of course handle it in any way you want.

def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [73]:
def create_new_twitter_table(table_name, creds):
    """
    Hace una nueva tabla de stocks según el ticker que le pongas
    
    Args:
        table_name(str): nombre del ticker
        conn(connection object): objeto de connección a la base de datos
    """
    conn = db_connection(creds)
    query = """CREATE TABLE public.twitter (id SERIAL,
    date date NOT NULL UNIQUE, 
    account text,
    text text, 
    PRIMARY KEY (id));""".format(table_name)
    download_data(conn, query)

def check_table_exists(tables_list, conn):
    """
    Checa si la lista de tablas existe en la base de datos
    
    Args:
        tables_list (list): lista con nombres de tablas
    """
    conn = db_connection(creds)
    query = """SELECT table_name FROM information_schema.tables;"""
    exist_tables = download_data(conn, query)['table_name'].tolist()
    
    intersect = set(tables_list) & set(exist_tables)
    res = list(filter(lambda x: x not in intersect, table_names))
    
    return res

def update_in_db(status, creds):
    """
    Inserta registros de twitter
    
    Args:
        status (twitter.status object): es el objeto de twitter status
        creds (dict): diccionario con las credenciales de la base de datos
    """
    conn = db_connection(creds)
    print(status.user._json.get('screen_name'))
    query = """INSERT INTO twitter (date, account, text, id_twitter) VALUES('{}','{}','{}',{});""".format(status.created_at.date(), status.user._json.get('screen_name'), status.full_text, status.id)
    download_data(conn, query)
    
def get_last_id(account_name, creds):
    """
    Trae el último id de la tabla 
    """
    conn = db_connection(creds)
    query = "SELECT MIN(id_twitter) from twitter where account='{}'".format(account_name.lower())
    df = download_data(conn,query)
    res = df.iloc[0][0]
    
    if res == None:
        res = 1
    
    return res

def update_in_bulk_db(status_list, creds):
    """
    Añade todos los tweets de una vez
    """
    tam_list = len(status_list)
    cuenta = status_list[0].user._json.get('screen_name')
    first_id = status_list[0].id
    last_id = status_list[tam_list-1].id
    
    print('se van a guardar {} tweets de la cuenta: {} [{}-{}]'.format(tam_list, cuenta, first_id, last_id))
    conn = db_connection(creds)
    data=["('{}','{}','{}',{})".format(status.created_at.date(), status.user._json.get('screen_name').lower(), status.full_text.replace("'","''"), status.id) for status in status_list]
    data_ready =",".join(data)
    query= """INSERT INTO twitter (date, account, text, id_twitter) VALUES {};""".format(data_ready)
    try:
        download_data(conn,query)
    except Exception as e:
        print('Error al insertar en la base'.format(e))

In [74]:
clean_accounts = [cuenta.replace('@', '') for cuenta in lista_cuentas['Cuentas']]

keys_count = int(len(tw_creds)/4)-1 #cuentas disponibles 
retry = 0 #número de retries que llevamos
max_retries = 4 #número de retries que queremos tener de la lista
api_key = 0 #el número de key en el que vamos
api = new_tw_api(api_key) #instanciamos una nueva api

for cuenta in clean_accounts:
    db_max_id = get_last_id(cuenta, creds) - 1 #checa cual es el id del último tweet de esa cuenta
    try:
        if db_max_id > 0:
            #si hay tweets saca desde el último hasta ese máximo
            #logging.warning('Extrayendo datos de: @{} desde tweet:{}'.format(cuenta, db_max_id))
            #tweets = [status for status in limit_handled(tweepy.Cursor(api.user_timeline, id=cuenta, since_id=db_max_id, tweet_mode='extended').items())]
            print("ya tiene twitts:@{}".format(cuenta))
        else:
            # si no hay tweets saca desde los que más pueda
            logging.warning('Extrayendo datos de nueva cuenta!: @{}'.format(cuenta))
            tweets = [status for status in limit_handled(tweepy.Cursor(api.user_timeline, id=cuenta, tweet_mode='extended').items())]                    
            update_in_bulk_db(tweets, creds)
            
            
    except tweepy.TweepError as e:
        print(e)
        print('api_key:{}/{}, retries:{}/{}'.format(api_key, keys_count, retry, max_retries))
        if api_key < keys_count and retry <= max_retries:
            api_key = api_key + 1 
            api = new_tw_api(api_key)
            clean_accounts.append(cuenta)
        elif api_key == keys_count:
            api_key = 0
            api = new_tw_api(api_key)
            retry = retry + 1
            clean_accounts.append(cuenta)
        else:
            print("Se acabaron las cuentas API")
            break

ya tiene twitts:@BMVMercados
ya tiene twitts:@VeporMasBanco
ya tiene twitts:@BIVAMX
ya tiene twitts:@El_Universal_Mx
ya tiene twitts:@OECD
ya tiene twitts:@AMAFORE_mx
ya tiene twitts:@MexicoComoVamos
ya tiene twitts:@NafinOficial
ya tiene twitts:@cofecemx
ya tiene twitts:@lopezobrador_
ya tiene twitts:@BWMX
ya tiene twitts:@Banxico
ya tiene twitts:@SE_mx
ya tiene twitts:@cnbvmx
ya tiene twitts:@SENER_mx
ya tiene twitts:@SATMX
ya tiene twitts:@InvestingMx
ya tiene twitts:@Investingcom
ya tiene twitts:@USTreasury
ya tiene twitts:@ecb
ya tiene twitts:@bankofengland
ya tiene twitts:@bankofcanada
ya tiene twitts:@wef
ya tiene twitts:@Forbes_Mexico
ya tiene twitts:@PDChina
ya tiene twitts:@IBTimes
ya tiene twitts:@realDonaldTrump
ya tiene twitts:@zerohedge
ya tiene twitts:@crypto
ya tiene twitts:@markets
ya tiene twitts:@business
ya tiene twitts:@GoldmanSachs
ya tiene twitts:@MorganStanley
ya tiene twitts:@stlouisfed
ya tiene twitts:@ReformedBroker
ya tiene twitts:@Carl_C_Icahn
ya tiene twit

In [ ]:
tweets = [status for status in limit_handled(tweepy.Cursor(api.user_timeline, id=cuenta, since_id=db_max_id, tweet_mode='extended').items())]

In [ ]:
print(a[0].created_at, type(a[0].created_at))
print(a[0].id, type(a[0].id))
print(a[0].id_str, type(a[0].id_str))
print(a[0].text, type(a[0].text))
print(a[0].truncated, type(a[0].truncated))
print(a[0].entities, type(a[0].entities))
print(a[0].source, type(a[0].source))
print(a[0].in_reply_to_status_id, type(a[0].in_reply_to_status_id))
print(a[0].in_reply_to_status_id_str, type(a[0].in_reply_to_status_id_str))
print(a[0].in_reply_to_user_id, type(a[0].in_reply_to_user_id))
print(a[0].in_reply_to_user_id_str, type(a[0].in_reply_to_user_id_str))
print(a[0].in_reply_to_screen_name, type(a[0].in_reply_to_screen_name))
print(a[0].user._json, type(a[0].user._json))
print(a[0].geo, type(a[0].geo))
print(a[0].coordinates, type(a[0].coordinates))
print(a[0].place, type(a[0].place))
print(a[0].contributors, type(a[0].contributors))
print(a[0].is_quote_status, type(a[0].is_quote_status))
print(a[0].retweet_count, type(a[0].retweet_count))
print(a[0].favorite_count, type(a[0].favorite_count))
print(a[0].favorited, type(a[0].favorited))
print(a[0].retweeted, type(a[0].retweeted))
print(a[0].possibly_sensitive, type(a[0].possibly_sensitive))
#print(type(a[0]._json))